In [55]:
#libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
#Project functions
from Project_functions import get_monthi,get_mean_temp

In [2]:
#setup
df = pd.read_csv("consumption.csv")
weather_avg = pd.read_csv('weather-avg.csv')
weather_min = pd.read_csv('weather-min.csv')
weather_max = pd.read_csv('weather-max.csv')
info = pd.read_csv('addinfo.csv')
weather_avg = weather_avg.set_index("meter_id")
weather_min = weather_min.set_index("meter_id")
weather_max = weather_max.set_index("meter_id")
info = info.set_index("meter_id")
weather_avg_sorted = weather_avg.reindex(df.iloc[:,0])
weather_min_sorted = weather_min.reindex(df.iloc[:,0])
weather_max_sorted = weather_max.reindex(df.iloc[:,0])
info_sorted = info.reindex(df.iloc[:,0])
brinfo=info_sorted['num_bedrooms']
values = {'num_bedrooms' : brinfo.mean()}
info_filled_br = info_sorted.fillna(value = values)
brinfo_filled = info_filled_br['num_bedrooms']

In [56]:
#these don't work out of the function file apperantly
def Eval_random_Split(features,npEn_con,split,model,print_res=True):
    X_train, X_test, y_train, y_test = train_test_split(
    features , npEn_con.reshape(-1,1), test_size=split, random_state=0)
    #print(X_train.shape, y_train.shape)
    #print(X_test.shape, y_test.shape)
    model.fit(X_train,y_train)
    if print_res:
        y_pred=model.predict(X_test)
        print('Coefficients: \n', model.coef_)
        print('Intercept: {}'.format(model.intercept_))
        # The mean squared error
        print('Mean squared error: %.8f' % mean_squared_error(y_test, y_pred))
        # The coefficient of determination: 1 is perfect prediction
        print('Coefficient of determination: %.8f' % r2_score(y_test, y_pred))
    return model

def Check_1D(features,npEn_con,split,model,print_res=True):
    #extract only the last month from the features
    features_lm=features[:,(features.shape[-1]-1)]
    features_lm=features_lm.reshape(-1,1)
    #use previously made function
    return Eval_random_Split(features_lm,npEn_con,split,model,print_res)

#z-normalization on data:
def normalise_features(np_features):
    #np_features_norm= np.zeros(shape = np_features.shape)
    np_features_norm=np_features
    if np_features.ndim==1:
        np_features_norm=(np_features-np.mean(np_features))/np.std(np_features)
    else: 
        for i in range(np_features.shape[1]):
            np_features_norm[:,i] = (np_features[:,i]-np.mean(np_features[:,i]))/np.std(np_features[:,i])
    return np_features_norm

def Eval_Kfold_Split(features,npEn_con,n_splits,model,print_res=True):
    kf = KFold(n_splits, shuffle=True)
    models_arr=[]
    MSE_arr=[]
    R2_arr=[]
    for train_index, test_index in kf.split(features): 
        cmodel=model
        cmodel.fit(features[train_index,],npEn_con[train_index])
        models_arr.append(cmodel)
        y_pred = model.predict(features[test_index,])
        MSE_arr.append(mean_squared_error(npEn_con[test_index,], y_pred));
        R2_arr.append(r2_score(npEn_con[test_index,], y_pred))
    if print_res:
        # The mean squared error
        print('Mean squared error:')
        print(np.mean(MSE_arr))
        # The coefficient of determination: 1 is perfect prediction
        print('Coefficient of determination: ' )
        print(np.mean(R2_arr))
    return models_arr

In [40]:
def sort_data(nim,nom):
    #nim: number of input months
    #nom: number of output months
    
    #CHECK INPUTS
    if nim<1:
        print("number of input months (nim) should be greater or equal to 1")
        return 0
    if nom<1:
        print("number of output months (nom) should be greater or equal to 1")
        return 0
    
    #DATA STORAGE
    En_con=[]    #current/future consumption (this is what we try to predict)
    prev_con=[] #previous consumtion
    #extra info
    temps=[]
    temps_min=[]
    temps_max=[]
    month_arr=[]
    br_arr=[]       #Bedroom info array
    br_arr_mean=[]
    dwt_arr=[]      #dwelling type array
    y = pd.get_dummies(info_sorted.dwelling_type, prefix='dwelling_type')
    
    #LOOP
    NaN_t=1200  #thershold
    for i in range(df.shape[0]): # loop over all users
        meter=df.iloc[i] # = row number i  
    
        #loop over all month from januari until month 14-nim-nom
        #if the month is higher then this we don't have enough data for the nim input and nom output variables
        #in the case of nim=1 nom=1 we get range(1,12) wich result in m going from 1 to 11 like we want
        for m in range (1,14-nim-nom): 
            #load current month
            bi, ei =get_monthi(m)    #bi (begin index) ei(end index)
            month=meter[bi:ei]       #data for the month m for the row user (row) i
            #load bedroom info
            row = brinfo[i:i+1]               #bedroom info with NaN values
            row_mean = brinfo_filled[i:i+1]  #bedroom info with mean values
            #Load dwelling type
            dew_type_row = y[i:i+1]
            
            # Count the amount of NaN's in current      
            n_NaN=month.isnull().sum() 

            # Check if months have numeric values otherwise discard the month for this user. 
            if n_NaN<NaN_t:
                for j in range(m,14-nim-nom):
                    #save input for month j,j+1,..,j+nim-1
                    input_months=np.zeros(nim)
                    output_months=np.zeros(nom)
                    for k in range(nim):
                        #load month for input j+k (k goes from 0 to nim-1)
                        bik,eik=get_monthi(j+k)
                        input_months[k]=meter[bik:eik].mean()
                    for k in range(nom):
                        #load month for input j+nim+k
                        bik,eik=get_monthi(j+nim+k)
                        output_months[k]=meter[bik:eik].mean()
                    #debug if statement checks for NaN in input month
                    if np.isnan(np.sum(input_months)):
                        print("input month has nan value for months=%i index=%i" %(j,i))
                        print(input_months)
                    if np.isnan(np.sum(output_months)):
                        print("Output months has nan value for months=%i index=%i" %(j,i))
                        print(output_months)
                    #store Energy consumption data
                    prev_con.append(input_months)
                    En_con.append(output_months)
                    #save extra information THE TEMPERATURE FOR ALL MONTHS EN CON MONTHS SHOULD BE SAVE THIS IS NOT THE CASE
                    temps.append(get_mean_temp(weather_avg_sorted.iloc[i:i+1],j+nim))
                    temps_min.append(get_mean_temp(weather_min_sorted.iloc[i:i+1],j+nim))
                    temps_max.append(get_mean_temp(weather_max_sorted.iloc[i:i+1],j+nim))
                    month_arr.append(j+nim)
                    #bedroom info
                    n_br=row[0]
                    br=np.zeros(5)
                    if ~np.isnan(n_br):
                        #print(n_br)
                        br[np.int(n_br-1)]=1
                    br_arr.append(br)
                    br_arr_mean.append(row_mean[0])
                    #dwelling type
                    dwt_arr.append(dew_type_row.to_numpy().ravel())
                        
                #break out of outer month loops since the inner month loop takes care of all months
                break
                
    # converting list to numpy arrays: 
    npEn_con=np.array(En_con)
    npprev_con=np.array(prev_con)
    nptemps=np.array(temps)
    nptemps_min=np.array(temps_min)
    nptemps_max=np.array(temps_max)
    npmonth=np.array(month_arr)
    npbr=np.array(br_arr)
    npbrmean=np.array(br_arr_mean)
    npdwt=np.array(dwt_arr)
    
    #This should probably be done differently 
    return npEn_con,npprev_con,nptemps,nptemps_min,nptemps_max,npmonth,npbr,npbrmean,npdwt
    

In [77]:
npEn_con,npprev_con,nptemps,nptemps_min,nptemps_max,npmonth,npbr,npbrmean,npdwt=sort_data(1,1)

In [31]:
yt = pd.get_dummies(info_sorted.dwelling_type, prefix='dwelling_type')
#print(yt)
i=1252
dew_type_row = yt[i:i+1]
print(dew_type_row.to_numpy())
print(dew_type_row.to_numpy().ravel())
#print(dew_type_row.isnull())
print(np.isnan(dew_type_row.to_numpy().ravel()))
print(np.any(np.isnan(dew_type_row.to_numpy().ravel())))

[[0 0 0 0 0]]
[0 0 0 0 0]
[False False False False False]
False


Test results new sort_data function

In [63]:
print(npEn_con.shape)
#1 month of previous data
Eval_random_Split(npprev_con,npEn_con,0.3,LinearRegression())
model=Eval_Kfold_Split(npprev_con,npEn_con,10,LinearRegression())
print("1 months")
np_features=np.column_stack(((npprev_con,nptemps,nptemps_min, nptemps_max, npbr)))
models_arr=Eval_Kfold_Split(np_features,npEn_con,10,LinearRegression())
#this seems to work let's try loading more than one input month

(17845, 1)
Coefficients: 
 [[1.01008592]]
Intercept: [0.00831724]
Mean squared error: 0.00323402
Coefficient of determination: 0.83505367
Mean squared error:
0.003332909297342171
Coefficient of determination: 
0.8342667822081762
1 months
Mean squared error:
0.00313942903726483
Coefficient of determination: 
0.8439381732263028


In [64]:
npEn_con2,npprev_con2,nptemps2,nptemps_min2,nptemps_max2,npmonth2,npbr2,npbrmean2,npdwt2=sort_data(2,1)
npEn_con3,npprev_con3,nptemps3,nptemps_min3,nptemps_max3,npmonth3,npbr3,npbrmean3,npdwt3=sort_data(3,1)

In [65]:
#2 months
print("2 months")
np_features=np.column_stack(((npprev_con2,nptemps2,nptemps_min2, nptemps_max2, npbr2)))
models_arr=Eval_Kfold_Split(np_features,npEn_con2,10,LinearRegression())
#3 months
print("\n 3 months")
np_features=np.column_stack(((npprev_con3,nptemps3,nptemps_min3, nptemps_max3, npbr3)))
models_arr=Eval_Kfold_Split(np_features,npEn_con3,10,LinearRegression())
#result seems more or less the same

2 months
Mean squared error:
0.0029432009360840445
Coefficient of determination: 
0.85368435061221

 3 months
Mean squared error:
0.002985080273165061
Coefficient of determination: 
0.8524227588394664


In [68]:
#test new dwelling type one hot encoding
print("1 months")
np_features=np.column_stack(((npprev_con,nptemps,nptemps_min, nptemps_max, npbr,npdwt)))
models_arr=Eval_Kfold_Split(np_features,npEn_con,10,LinearRegression())
print("normalised")
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n")
print("2 months")
np_features=np.column_stack(((npprev_con2,nptemps2,nptemps_min2, nptemps_max2, npbr2,npdwt2)))
models_arr=Eval_Kfold_Split(np_features,npEn_con2,10,LinearRegression())
print("normalised")
np_features_norm=normalise_features(np_features)
npEn_con2_norm=normalise_features(npEn_con2)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con2_norm,10,LinearRegression())

1 months
Mean squared error:
0.15591067957611787
Coefficient of determination: 
0.843619962891316
normalised
Mean squared error:
0.15584969121687714
Coefficient of determination: 
0.8439478506141658


2 months
Mean squared error:
0.14688738960621736
Coefficient of determination: 
0.8527429343702894
normalised
Mean squared error:
0.14684242256307706
Coefficient of determination: 
0.8524045271628602


In [161]:
np_features=np.column_stack(((npprev_con,nptemps)) 
#only the npprev_con and nptemps and npmonth feature seem important 
#the other feature do not influence the result greatly
#np_features=npprev_con
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())

Mean squared error:
0.16557167639697662
Coefficient of determination: 
0.8333965681188575


### More months to predict

In [69]:
npEn_con22,npprev_con22,nptemps22,nptemps_min22,nptemps_max22,npmonth22,npbr22,npbrmean22,npdwt22=sort_data(2,2)

In [72]:
#tests
print(npEn_con22.shape)
np_features=np.column_stack(((npprev_con22,nptemps22,nptemps_min22, nptemps_max22, npbr22,npdwt22)))
npEn_con=npEn_con22
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())

(12164, 2)
Mean squared error:
0.2545935626826167
Coefficient of determination: 
0.7447883683072353


In [75]:
np_features=np.column_stack(((npprev_con22,nptemps22,nptemps_min22, nptemps_max22, npbr22,npdwt22)))
npEn_con=npEn_con22[:,0]
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())

Mean squared error:
0.1721346661258377
Coefficient of determination: 
0.8283934488819575


As you'd expect the error has increased but is still not terrible. <br>
In the second blok we also see that this is not due to the reduced data set.

In [162]:
npEn_con12,npprev_con12,nptemps12,nptemps_min12,nptemps_max12,npmonth12,npbr12,npbrmean12,npdwt12=sort_data(1,2)
npEn_con13,npprev_con13,nptemps13,nptemps_min13,nptemps_max13,npmonth13,npbr13,npbrmean13,npdwt13=sort_data(1,3)
npEn_con16,npprev_con16,nptemps16,nptemps_min16,nptemps_max16,npmonth16,npbr16,npbrmean16,npdwt16=sort_data(1,6)
npEn_con23,npprev_con23,nptemps23,nptemps_min23,nptemps_max23,npmonth23,npbr23,npbrmean23,npdwt23=sort_data(2,3)
npEn_con26,npprev_con26,nptemps26,nptemps_min26,nptemps_max26,npmonth26,npbr26,npbrmean26,npdwt26=sort_data(2,6)
npEn_con32,npprev_con32,nptemps32,nptemps_min32,nptemps_max32,npmonth32,npbr32,npbrmean32,npdwt32=sort_data(3,2)
npEn_con33,npprev_con33,nptemps33,nptemps_min33,nptemps_max33,npmonth33,npbr33,npbrmean33,npdwt33=sort_data(3,3)
npEn_con36,npprev_con36,nptemps36,nptemps_min36,nptemps_max36,npmonth36,npbr36,npbrmean36,npdwt36=sort_data(3,6)

In [172]:
print("################## \n1 2")
print(npEn_con12.shape)
np_features=np.column_stack(((npprev_con12,nptemps12,nptemps_min12, nptemps_max12, npbr12,npdwt12)))
npEn_con1=npEn_con12
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n1 output month check")
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm[:,0],10,LinearRegression())
print("################## \n1 3")
print(npEn_con13.shape)
np_features=np.column_stack(((npprev_con13,nptemps13,nptemps_min13, nptemps_max13, npbr13,npdwt13)))
npEn_con1=npEn_con13
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n1 output month check")
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm[:,0],10,LinearRegression())
print("################## \n1 6")
print(npEn_con16.shape)
np_features=np.column_stack(((npprev_con16,nptemps16,nptemps_min16, nptemps_max16, npbr16,npdwt16)))
npEn_con1=npEn_con16
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n1 output month check")
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm[:,0],10,LinearRegression())

################## 
1 2
(14868, 2)
Mean squared error:
0.2725120205426432
Coefficient of determination: 
0.7271512042549124

1 output month check
Mean squared error:
0.1768928917880479
Coefficient of determination: 
0.8231367777498926
################## 
1 3
(12164, 3)
Mean squared error:
0.32523130433900993
Coefficient of determination: 
0.6749415040083415

1 output month check
Mean squared error:
0.12202860367652764
Coefficient of determination: 
0.8775988342610285
################## 
1 6
(5672, 6)
Mean squared error:
0.334124708181922
Coefficient of determination: 
0.6652504179752872

1 output month check
Mean squared error:
0.09457971308183746
Coefficient of determination: 
0.9038447935697518


In [175]:
print("################## \n2 2")
print(npEn_con22.shape)
np_features=np.column_stack(((npprev_con22,nptemps22,nptemps_min22, nptemps_max22, npbr22,npdwt22)))
npEn_con1=npEn_con22
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n1 output month check")
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm[:,0],10,LinearRegression())
print("################## \n2 3")
print(npEn_con23.shape)
np_features=np.column_stack(((npprev_con23,nptemps23,nptemps_min23, nptemps_max23, npbr23,npdwt23)))
npEn_con1=npEn_con23
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n1 output month check")
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm[:,0],10,LinearRegression())
print("################## \n2 6")
print(npEn_con26.shape)
np_features=np.column_stack(((npprev_con26,nptemps26,nptemps_min26, nptemps_max26, npbr26,npdwt26)))
npEn_con1=npEn_con26
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n1 output month check")
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm[:,0],10,LinearRegression())

################## 
2 2
(12164, 2)
Mean squared error:
0.2547782381954041
Coefficient of determination: 
0.7453798124528687

1 output month check
Mean squared error:
0.17226352947837378
Coefficient of determination: 
0.8282204125757536
################## 
2 3
(9729, 3)
Mean squared error:
0.3090429970824544
Coefficient of determination: 
0.6913264348624913

1 output month check
Mean squared error:
0.12534172237449653
Coefficient of determination: 
0.8744231977973274
################## 
2 6
(4050, 6)
Mean squared error:
0.31133404957524524
Coefficient of determination: 
0.6878302142429276

1 output month check
Mean squared error:
0.10102070467558018
Coefficient of determination: 
0.8962590040032467


In [174]:
print("################## \n3 2")
print(npEn_con32.shape)
np_features=np.column_stack(((npprev_con32,nptemps32,nptemps_min32, nptemps_max32, npbr32,npdwt32)))
npEn_con1=npEn_con32
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n1 output month check")
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm[:,0],10,LinearRegression())
print("################## \n3 3")
print(npEn_con33.shape)
np_features=np.column_stack(((npprev_con33,nptemps33,nptemps_min33, nptemps_max33, npbr33,npdwt33)))
npEn_con1=npEn_con33
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n1 output month check")
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm[:,0],10,LinearRegression())
print("################## \n3 6")
print(npEn_con36.shape)
np_features=np.column_stack(((npprev_con36,nptemps36,nptemps_min36, nptemps_max36, npbr36,npdwt36)))
npEn_con1=npEn_con36
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n1 output month check")
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm[:,0],10,LinearRegression())

################## 
3 2
(9729, 2)
Mean squared error:
0.24923874401934581
Coefficient of determination: 
0.7496086086441274

1 output month check
Mean squared error:
0.173735233847974
Coefficient of determination: 
0.8252875388861483
################## 
3 3
(7565, 3)
Mean squared error:
0.298855986374971
Coefficient of determination: 
0.6997284296582352

1 output month check
Mean squared error:
0.12552079911282762
Coefficient of determination: 
0.8751380719257768
################## 
3 6
(2699, 6)
Mean squared error:
0.2891669846974029
Coefficient of determination: 
0.706913138816124

1 output month check
Mean squared error:
0.10423094056249951
Coefficient of determination: 
0.8935145599540361
